# Mongo Function

In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from bson.objectid import ObjectId
import time

In [ ]:
load_dotenv()

MONGO_URI = os.getenv('MONGO_URI')
client = MongoClient(MONGO_URI)

db = client["ImaginAI"]
collection = db["stock_data"]

## Get Home Images

In [ ]:
def GetHomeImages(collection=None,cursor=0,size=50):
    if collection is None:
        return {}
    query = {}
    documents = list(collection.find(query).skip(cursor).limit(size))
    url_list = [doc["image_url_list"][0].split("/")[-1] for doc in documents]
    next_cursor = cursor + len(url_list)
    return {"url_list":url_list,"next_cursor":next_cursor}

In [ ]:
GetHomeImages(collection,0,10)

## Search Prompt

In [ ]:
def SearchPrompt(collection=None,cursor=0,size=50,prompt=""):
    if len(prompt.split(" ")) > 3:
        return {"response" : "Too Many keywords"}
    if prompt == "":
        return {"response" : "No Keywords provided"}
    query = [
        {
            '$search': {
                'index': 'search_prompt',
                'text': {
                    'query': f'{prompt}',
                    'path': {
                        'wildcard': '*'
                    },
                    'fuzzy': {}
                }
            }
        },
        {
            '$limit': size  # Number of documents to retrieve
        },
        {
            '$skip': cursor  # Number of documents to skip
        }
    ]
    result = collection.aggregate(query)

    return [doc for doc in result]

In [72]:
documents = SearchPrompt(collection,0,50,"women")

In [73]:
for doc in documents:
    print(doc['_id'],doc['prompt'])

641d6e6b92e339fcd4169e78 Rainbow-colored haired woman
641d721b0e8f6e83ca727c9e A woman with dahlia flowers
641d6e4592e339fcd4169e6a A woman with long hair
641d6e3692e339fcd4169e64 A woman with colored hair
641d6e3092e339fcd4169e62 A woman with colorful hair
641d6e6092e339fcd4169e74 A woman with long colorful hair
641d6dfb92e339fcd4169e4e A woman with butterflies on her face
641d71bd0e8f6e83ca727c78 A woman with flowers on her head
641d817c0e8f6e83ca728260 A woman looking at a butterfly
641d722a0e8f6e83ca727ca4 A woman with daisy flowers on her head
641d72070e8f6e83ca727c96 Cherry blossoms adorn the head of a woman
641d71c70e8f6e83ca727c7c A woman wears anemone flowers on her head
641d72210e8f6e83ca727ca0 A woman with dahlia flowers in her hair
641d72430e8f6e83ca727cae A woman lying down with her eyes closed
641d72340e8f6e83ca727ca8 A woman with daisy flowers on her face
641d6e6692e339fcd4169e76 The hair of a woman with rainbow colors
641d71e50e8f6e83ca727c88 A woman wears azalea flower

## Search Image

In [49]:

def SearchImage(collection=None,vector=[]):
    query = [
                {
                        "$search": {
                            "index": "search_image",
                            "knnBeta": {
                            "vector": vector,
                            "path": "content_embedding",
                            "k": 2000
                            }
                        }
                },
                {
                "$skip": 0
                },
                {
                "$limit": 24
                }
            ]
    result = collection.aggregate(query)
    return [doc for doc in result]

In [74]:
start = time.time()
vector = collection.find({"_id":ObjectId("641d6e4592e339fcd4169e6a")},{"content_embedding"})[0]['content_embedding']
documents = SearchImage(collection,vector)
print(time.time() - start)

0.2622547149658203


## Inference Image Similarity

In [75]:
import requests
from io import BytesIO
from joblib import Parallel,delayed
from PIL import Image

def download_image(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    return image

image_urls = [doc['image_url_list'][0] for doc in documents]
images = Parallel(n_jobs=24)(delayed(download_image)(url) for url in image_urls)

In [71]:
from PIL import Image
import numpy as np

# Create a blank canvas for the grid
canvas_width = 600  # Adjust the canvas dimensions as desired
canvas_height = 400
canvas = Image.new('RGB', (canvas_width, canvas_height), (255, 255, 255))

# Load and resize the images
 # Replace with the respective image paths or URLs
reImages = []
for img in images:
    img = img.resize((canvas_width // 6, canvas_height // 4))  # Divide canvas dimensions by grid size (4x6)
    reImages.append(img)

# Paste the images onto the canvas in a grid pattern
x_offset = 0
y_offset = 0
for img in reImages:
    canvas.paste(img, (x_offset, y_offset))
    x_offset += img.width
    if x_offset >= canvas_width:
        x_offset = 0
        y_offset += img.height

# Display or save the resulting grid image
# canvas.show()  # Display the image
canvas.save("image_grid4.jpg")  # Save the image to a file
